In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from prophet import Prophet
import plotly.express as px
import plotly.figure_factory as ff

import warnings
from soupsieve import select
warnings.filterwarnings('ignore')
1
# Shows all columns 
pd.set_option("display.max_columns", None)

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
from prophet import Prophet


In [5]:
# C:\Users\Ramshankar\OneDrive - iLink Systems Inc\Documents\Costco\Dataset\Final Output\Weekly
    
    
    
data1 = pd.read_excel("C:\\Users\\Ramshankar\\OneDrive - iLink Systems Inc\\Documents\\Costco\\Dataset\\Final Output\\Weekly\\Data_Weekly.xlsx")

In [6]:
data1

,Location,Item,Date,Quantity,Amount
0,FREMONT,CHICKEN XXXXX - O,2020-01-05,37.02338,184.74667
1,FREMONT,CHICKEN XXXXX - O,2020-01-12,41.88978,209.03000
2,FREMONT,CHICKEN XXXXX - O,2020-01-19,86.10121,429.64500
3,FREMONT,CHICKEN XXXXX - O,2020-01-26,63.83407,318.53200
4,FREMONT,CHICKEN XXXXX - O,2020-02-02,45.32098,226.15167
...,...,...,...,...,...
2122,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,2022-10-30,113.51292,146.43167
2123,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,2022-11-06,97.40439,125.65167
2124,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,2022-11-13,93.24030,120.28000
2125,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,2022-11-20,94.38113,121.75167


In [7]:
data2 = data1[['Date', 'Quantity', 'Location', 'Item']]

In [8]:
data3= data2[(data2['Location']== 'FREMONT') & (data2['Item']== 'CHICKEN XXXXX - O') ]

In [9]:
train = data3[data3['Date']<= '2022-05-01']
test = data3[data3['Date']>= '2022-05-01']

In [10]:
train.drop(['Location', 'Item'], axis=1, inplace=True)

In [11]:
train

,Date,Quantity
0,2020-01-05,37.02338
1,2020-01-12,41.88978
2,2020-01-19,86.10121
3,2020-01-26,63.83407
4,2020-02-02,45.32098
...,...,...
117,2022-04-03,45.78255
118,2022-04-10,71.82971
119,2022-04-17,44.06711
120,2022-04-24,43.08514


In [12]:
train= train.rename(columns= {'Date': 'ds', 'Quantity': 'y'})

In [13]:
model = Prophet()
model.fit(train)

19:22:21 - cmdstanpy - INFO - Chain [1] start processing
19:22:22 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
test1= test.drop(['Quantity', 'Location', 'Item'], axis=1)
test1= test1.rename(columns= {'Date': 'ds'})
test1.head()

,ds
121,2022-05-01
122,2022-05-08
123,2022-05-15
124,2022-05-22
125,2022-05-29


In [15]:
forecasted = model.predict(test1)

In [16]:
forecasted.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-05-01,44.66308,7.43673,72.95703,44.66308,44.66308,-4.17530,-4.17530,-4.17530,-4.17530,-4.17530,-4.17530,0.00000,0.00000,0.00000,40.48777
1,2022-05-08,44.09945,14.47935,79.54339,44.09940,44.09945,2.91594,2.91594,2.91594,2.91594,2.91594,2.91594,0.00000,0.00000,0.00000,47.01540
2,2022-05-15,43.53583,17.99624,86.31278,43.51823,43.55075,7.34751,7.34751,7.34751,7.34751,7.34751,7.34751,0.00000,0.00000,0.00000,50.88334
3,2022-05-22,42.97221,15.37163,80.04846,42.93160,43.00681,4.93326,4.93326,4.93326,4.93326,4.93326,4.93326,0.00000,0.00000,0.00000,47.90547
4,2022-05-29,42.40858,8.52867,71.54343,42.34283,42.46748,-1.04548,-1.04548,-1.04548,-1.04548,-1.04548,-1.04548,0.00000,0.00000,0.00000,41.36310


In [17]:
forecasted_df= forecasted[['ds', 'yhat']]

In [23]:
data4= data3.rename(columns= {'Date':'ds', 'Quantity': 'y'})

In [24]:
data_merge= pd.merge(data4,forecasted_df, on= ['ds'], how ='inner' )

In [25]:
data_merge

,ds,y,Location,Item,yhat
0,2022-05-01,55.80034,FREMONT,CHICKEN XXXXX - O,40.48777
1,2022-05-08,41.93823,FREMONT,CHICKEN XXXXX - O,47.01540
2,2022-05-15,65.49999,FREMONT,CHICKEN XXXXX - O,50.88334
3,2022-05-22,27.73074,FREMONT,CHICKEN XXXXX - O,47.90547
4,2022-05-29,19.67863,FREMONT,CHICKEN XXXXX - O,41.36310
5,2022-06-05,30.85576,FREMONT,CHICKEN XXXXX - O,38.71834
6,2022-06-12,21.52087,FREMONT,CHICKEN XXXXX - O,42.98219
7,2022-06-19,30.11986,FREMONT,CHICKEN XXXXX - O,48.71102
8,2022-06-26,20.83005,FREMONT,CHICKEN XXXXX - O,47.82405
9,2022-07-03,18.73313,FREMONT,CHICKEN XXXXX - O,38.81300


In [26]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [27]:
mean_squared_error(data_merge.y, data_merge.yhat)

295.6236766235743

In [28]:
import math 

math.sqrt(mean_squared_error(data_merge.y, data_merge.yhat))

17.19371037977476

In [30]:
 from sklearn.metrics import mean_absolute_percentage_error

In [31]:
MAPE_Score= mean_absolute_percentage_error(data_merge.y, data_merge.yhat).round(2)
print('MAPE_Score:', MAPE_Score)

MAPE_Score: 0.45


### All in one 

In [32]:
data2

,Date,Quantity,Location,Item
0,2020-01-05,37.02338,FREMONT,CHICKEN XXXXX - O
1,2020-01-12,41.88978,FREMONT,CHICKEN XXXXX - O
2,2020-01-19,86.10121,FREMONT,CHICKEN XXXXX - O
3,2020-01-26,63.83407,FREMONT,CHICKEN XXXXX - O
4,2020-02-02,45.32098,FREMONT,CHICKEN XXXXX - O
...,...,...,...,...
2122,2022-10-30,113.51292,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN
2123,2022-11-06,97.40439,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN
2124,2022-11-13,93.24030,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN
2125,2022-11-20,94.38113,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN


In [33]:
for i in data2['Location'].unique():
    df= data2[data2['Location']==i]
#     print(df)
    for j in df['Item'].unique():
        item_df= df[df['Item']==j]
#         print(item_df)
        
        print("\n")
        print("Preparing data for modelling")
        item_df= item_df.drop(['Location', 'Item'], axis=1)
        item_df= item_df.rename(columns={'Date':'ds', 'Quantity': 'y'})
        
        train = item_df[item_df['ds']<= '2022-05-01']
        test = item_df[item_df['ds']>= '2022-05-01']
        test1= test.drop(['y'], axis=1)
        
        print("\n")
        print(f"Model Training initiated for {i} and {j}")
        model = Prophet()
        model.fit(train)
        
        print("Model forecasting")
        forecasted = model.predict(test1)
        forecasted_df= forecasted[['ds', 'yhat']]
        
        print("Merging forecasting and test data")
        data_merge= pd.merge(test,forecasted_df, on= ['ds'], how ='inner')
        
#         print("\n")
        print(f"Tranining complted for {i} and {j}")
        print("Finding error matrices MSE and RMSE")
        MSE_Score = mean_squared_error(data_merge.y, data_merge.yhat).round(2)
        print(f'Train Score {i}-{j}: %.2f MSE' % (MSE_Score))
        
        RMSE_Score = math.sqrt(mean_squared_error(data_merge.y, data_merge.yhat))
        print(f'Train Score {i}-{j}: %.2f RMSE' % (RMSE_Score))
        
        from sklearn.metrics import mean_absolute_percentage_error

        MAPE_Score= mean_absolute_percentage_error(data_merge.y, data_merge.yhat).round(2)
        print('MAPE_Score:', MAPE_Score)
        
        
        col_names =  ['MSE_prophet', 'RMSE_prophet', 'MAPE_prophet']
        Error_matric= pd.DataFrame(columns = col_names)
        
        my_out = {'MSE_prophet': MSE_Score, 'RMSE_prophet': RMSE_Score, 'MAPE_prophet': MAPE_Score }
        Error_matric.loc[len(Error_matric)] = my_out 
        
        Error_matric['Location']=i
        Error_matric['Item']=j
        
#         C:\Users\Ramshankar\OneDrive - iLink Systems Inc\Documents\Costco\Code\ModelOut\All_out\Weekly\Prophet
        print(f'Writing error matrices of {i} and {j}')
        print("\n")
        Error_output = "ModelOut\All_out\Weekly\Prophet\Error\Output_Error -" +i+"-"+j+".xlsx"
        Error_matric.to_excel(Error_output, index= False)
        
        #Saving Results 
        forecasted_df['Location']= i
        forecasted_df['Item']=j
        
        print("\n")
        print(f'Writing predicted results of {i} and {j}')
        predction_output = "ModelOut\All_out\Weekly\Prophet\Output\Output -" +i+"-"+j+".xlsx"
        forecasted_df.to_excel(predction_output, index= False)
        
        
        

19:34:09 - cmdstanpy - INFO - Chain [1] start processing
19:34:09 - cmdstanpy - INFO - Chain [1] done processing




Preparing data for modelling


Model Training initiated for FREMONT and CHICKEN XXXXX - O
Model forecasting


19:34:09 - cmdstanpy - INFO - Chain [1] start processing


Merging forecasting and test data
Tranining complted for FREMONT and CHICKEN XXXXX - O
Finding error matrices MSE and RMSE
Train Score FREMONT-CHICKEN XXXXX - O: 295.62 MSE
Train Score FREMONT-CHICKEN XXXXX - O: 17.19 RMSE
MAPE_Score: 0.45
Writing error matrices of FREMONT and CHICKEN XXXXX - O




Writing predicted results of FREMONT and CHICKEN XXXXX - O


Preparing data for modelling


Model Training initiated for FREMONT and DRUMSTICKS NNNN


19:34:09 - cmdstanpy - INFO - Chain [1] done processing


...

Merging forecasting and test data
Tranining complted for RICHMOND CA and FILLET QQQQ
Finding error matrices MSE and RMSE
Train Score RICHMOND CA-FILLET QQQQ: 813.07 MSE
Train Score RICHMOND CA-FILLET QQQQ: 28.51 RMSE
MAPE_Score: 0.34
Writing error matrices of RICHMOND CA and FILLET QQQQ




Writing predicted results of RICHMOND CA and FILLET QQQQ


Preparing data for modelling


Model Training initiated for S SAN FRANCISCO BUS CTR and BEEF OOOO - GT


19:34:15 - cmdstanpy - INFO - Chain [1] done processing


Model forecasting


19:34:16 - cmdstanpy - INFO - Chain [1] start processing
19:34:16 - cmdstanpy - INFO - Chain [1] done processing


Merging forecasting and test data
Tranining complted for S SAN FRANCISCO BUS CTR and BEEF OOOO - GT
Finding error matrices MSE and RMSE
Train Score S SAN FRANCISCO BUS CTR-BEEF OOOO - GT: 3469.55 MSE
Train Score S SAN FRANCISCO BUS CTR-BEEF OOOO - GT: 58.90 RMSE
MAPE_Score: 0.15
Writing error matrices of S SAN FRANCISCO BUS CTR and BEEF OOOO - GT




Writing predicted results of S SAN FRANCISCO BUS CTR and BEEF OOOO - GT


Preparing data for modelling


Model Training initiated for S SAN FRANCISCO BUS CTR and DRUMSTICKS NNNN
Model forecasting
Merging forecasting and test data
Tranining complted for S SAN FRANCISCO BUS CTR and DRUMSTICKS NNNN
Finding error matrices MSE and RMSE
Train Score S SAN FRANCISCO BUS CTR-DRUMSTICKS NNNN: 1287.69 MSE
Train Score S SAN FRANCISCO BUS CTR-DRUMSTICKS NNNN: 35.88 RMSE
MAPE_Score: 0.17
Writing error matrices of S SAN FRANCISCO BUS CTR and DRUMSTICKS NNNN




Writing predicted results of S SAN FRANCISCO BUS CTR and DRUMSTICKS NNNN


In [35]:
import pandas as pd
import os
import glob


In [36]:
path = 'C:\\Users\\Ramshankar\\OneDrive - iLink Systems Inc\\Documents\\Costco\\Code\\ModelOut\\All_out\\Weekly\Prophet\\Output\\'
out_files = glob.glob(os.path.join(path, "*.xlsx"))

dfs= []


for f in out_files:
    dfs.append(pd.read_excel(f))
df_out=pd.concat(dfs, ignore_index=True)

In [37]:
df_out

,ds,yhat,Location,Item
0,2022-05-01,40.48777,FREMONT,CHICKEN XXXXX - O
1,2022-05-08,47.01540,FREMONT,CHICKEN XXXXX - O
2,2022-05-15,50.88334,FREMONT,CHICKEN XXXXX - O
3,2022-05-22,47.90547,FREMONT,CHICKEN XXXXX - O
4,2022-05-29,41.36310,FREMONT,CHICKEN XXXXX - O
...,...,...,...,...
429,2022-10-30,112.89371,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN
430,2022-11-06,103.87659,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN
431,2022-11-13,96.10625,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN
432,2022-11-20,94.61986,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN


In [38]:
data2.shape

(2127, 4)

In [39]:
df_out= df_out.rename(columns= {'ds': 'Date', 'yhat': 'Forecasted_Qty'})
data2= data2.rename(columns= {'Mv_Qty': 'Moving_Avg_Qty'})

In [40]:
data_merge= pd.merge(data2,df_out, on= ['Date', 'Location', 'Item'], how= 'left')

In [41]:
data_merge

,Date,Quantity,Location,Item,Forecasted_Qty
0,2020-01-05,37.02338,FREMONT,CHICKEN XXXXX - O,NaN
1,2020-01-12,41.88978,FREMONT,CHICKEN XXXXX - O,NaN
2,2020-01-19,86.10121,FREMONT,CHICKEN XXXXX - O,NaN
3,2020-01-26,63.83407,FREMONT,CHICKEN XXXXX - O,NaN
4,2020-02-02,45.32098,FREMONT,CHICKEN XXXXX - O,NaN
...,...,...,...,...,...
2122,2022-10-30,113.51292,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,112.89371
2123,2022-11-06,97.40439,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,103.87659
2124,2022-11-13,93.24030,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,96.10625
2125,2022-11-20,94.38113,S SAN FRANCISCO BUS CTR,DRUMSTICKS NNNN,94.61986


In [42]:
data_merge['Forecasted_Qty'] =data_merge['Forecasted_Qty'].fillna(0)

In [43]:
data_merge.to_excel('Prophet_Forecast_Weekly_All.xlsx', index=False)

In [44]:
path = 'C:\\Users\\Ramshankar\\OneDrive - iLink Systems Inc\\Documents\\Costco\\Code\\ModelOut\\All_out\\Weekly\Prophet\\Error\\'
error_files = glob.glob(os.path.join(path, "*.xlsx"))

len(error_files)

dfs_err= []


for f in error_files:
    dfs_err.append(pd.read_excel(f))
dfs_err=pd.concat(dfs_err, ignore_index=True)



In [45]:
dfs_err.to_excel('Error_Output_all_Prophet_Weekly.xlsx', index=False)